# Fall data analysis
This notebook contains the algorithm to analyze fall data from a sensor



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pylab as plt
import os
from matplotlib.widgets import Slider, Button
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import *
from sklearn import metrics

# To display more rows if required
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

#constants
csv_headers = ['timestamp','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']
data_headers = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']

fall_scene_tags_headers = ['filename','scene','event']

fall_directory = './fall_data/'
non_fall_directory = './non_fall_data/'



In [2]:
# Functions to read files

# Get a dataframe from a csv
def get_dataframe(filename):
    input_data = pd.read_csv(filename, sep=',' , names= csv_headers)
    return input_data

# Get a dataframe containing all the filenames and its tag and event
def get_file_tags(filename):
    input_data = pd.read_csv(filename, sep=',' , names= fall_scene_tags_headers)
    return input_data

In [3]:
#functions to analize dataframe

def prepare_df(df ,window_size, init_zones , fall_zones , reference_zone):
    for cell in data_headers:
        df[f'{cell}d'] = df[cell] - df[cell].shift(window_size)
    
    for cell in data_headers:
        df[f'{cell}dt'] = 0
        for ci in range(1, window_size + 1):
            df[f'{cell}dt'] += df[cell] - df[cell].shift(ci)
        df[f'{cell}dt'] /= window_size

    iz = 0;
    for zone in init_zones:
        colname = f'iza{iz}'
        df[colname] = df[zone].mean(axis=1)
        iz = iz + 1
    
    iz = 0;
    for zone in fall_zones:
        colname = f'fza{iz}'
        df[colname] = df[zone].mean(axis=1)
        iz = iz + 1
        
    iz = 0;
    for zone in init_zones:
        colname = f'izda{iz}'
        zone_d = [f'{z}d' for z in zone]
        df[colname] = df[zone_d].mean(axis=1)
        iz = iz + 1
    
    iz = 0;
    for zone in fall_zones:
        colname = f'fzda{iz}'
        zone_d = [f'{z}d' for z in zone]
        df[colname] = df[zone_d].mean(axis=1)
        iz = iz + 1
    
    df['rza'] = df[reference_zone].mean(axis=1)
    zone_d = [f'{z}d' for z in reference_zone]
    df['rzda'] = df[zone_d].mean(axis=1)
    
    #for cell in data_headers:
        #del df[f'{cell}d']

def analize_df(df ,window_size , init_zones , fall_zones , reference_zone , heatTransferThreshold = 0.2, heatAverageThreshold = 1.0 ,deltaThreshold = 0.4, clean = False , remove_temps = False):
    prepare_df(df, window_size , init_zones ,fall_zones , reference_zone )
    i = 0
    f = 0
    any_col = []
    for iz in init_zones:
        f = 0
        for fz in fall_zones:
            newcolname = f'iz{i}xfz{f}'
            iza , izda , fza , fzda , rza , rzda = df[f'iza{i}'] , df[f'izda{i}'],df[f'fza{f}'] , df[f'fzda{f}'] , df['rza'] , df['rzda'] 
            
            test1 = (iza < (rza + heatAverageThreshold )) & ((rza + heatAverageThreshold ) < fza)
            test2 = (izda <= -deltaThreshold ) & (deltaThreshold <= fzda)
            
            heatTransferPerc = izda / fzda
            
            test3 = heatTransferPerc < 0
            
            heatTransferPerc = heatTransferPerc.abs()
            
            test4 = ((1 - heatTransferThreshold) <= heatTransferPerc) & (heatTransferPerc <= (1 + heatTransferThreshold ))
            
            new_serie =  test1 & test2 & test3 & test4
            any_col.append(new_serie)
            df[newcolname] = new_serie
            f = f + 1
        i = i + 1
    df['any'] = False
    for serie in any_col:
        df['any'] |= serie
    
    if clean:
        i = 0
        f = 0
        for cell in data_headers:
            del df[f'{cell}d']
            del df[f'{cell}dt']
        for iz in init_zones:
            del df[f'iza{i}']
            del df[f'izda{i}']
            i = i + 1;
        for fz in fall_zones:
            del df[f'fza{f}']
            del df[f'fzda{f}']
            f = f + 1;
        del df['rza']
        del df['rzda']
    
    
    return

## Get list of files

In [4]:
fall_files = os.listdir(fall_directory)
non_fall_files = os.listdir(non_fall_directory)

print( 'fall files: ' , len(fall_files))
print( 'non-fall files: ' , len(non_fall_files))

fall files:  238
non-fall files:  246


## Get filename tags

In [5]:
overall_df = get_file_tags('fall_scene_tag.csv')
overall_df = overall_df.set_index('filename')

filecount = len(overall_df)

## Algorithm parameters

**Window_size:** this variable represents

**HeatTransferThreshold:** this variable represents

**HeatAverageThreshold:** this variable represents

**DeltaThreshold:** this variable represents

**Clean:** this variable represents

**Remove_temps:** this variable represents

In [6]:
#Algorithm parameters

Window_size = 10
HeatTransferThreshold = 0.3
HeatAverageThreshold = 1.0
DeltaThreshold = 0.35
Clean = True
Remove_temps = True

scenes = [
    #lateral fall #0
    {
        'name' : 'lateral scene detection',
        'init_z' : [
            ['1','5','9' ],
            ['2','6','10'],
            ['3','7','11'],
            ['4','8','12']
            ],

        'fall_z' : [
            ['13','14','15','16']
            ],
        'ref_z' : data_headers,
    },
    #frontal fall #1
    {
        'name': 'frontal scene detection',
        'init_z' : [
            ['6' , '10' ],
            ['7' , '11' ],
            ],

        'fall_z' : [
            ['14'],
            ['15']
            ],
        'ref_z' : data_headers,
    },
    #diagonal fall right -> left   #2
    {
        'name':'diagonal scene detection',
        'init_z' : [
            ['3' , '6', '7', '8','11' ],
            ],

        'fall_z' : [
            ['6', '9', '10','11' , '14'],
            ],
        'ref_z' : data_headers,
    },
    #diagonal fall left -> right   #3
    {
        'name':'diagonal2 scene detection',
        'init_z' : [
            ['5' , '6', '9', '10' ],
            ],
        'fall_z' : [
            ['11' , '12' , '15' , '16'],
            ],
        'ref_z' : data_headers,
                
    },
]

In [7]:
fall_results = [] 
for fall_file in fall_files:
    df = get_dataframe(fall_directory + fall_file)
    res = {}
    
    for scene in scenes:
        dfclone = pd.DataFrame(df)
        analize_df(dfclone , Window_size , scene['init_z'] , scene['fall_z'] , scene['ref_z'], HeatTransferThreshold, HeatAverageThreshold,DeltaThreshold, Clean,Remove_temps)
        #res[scene['name']] = dfclone['any'].any()
        overall_df.loc[fall_file,scene['name']] = dfclone['any'].any() * 1
    #fall_results.append(res)

non_fall_results = [] 
for fall_file in non_fall_files:
    df = get_dataframe(non_fall_directory + fall_file)
    res = {}
    
    for scene in scenes:
        dfclone = pd.DataFrame(df)
        analize_df(dfclone , Window_size , scene['init_z'] , scene['fall_z'] , scene['ref_z'], HeatTransferThreshold, HeatAverageThreshold,DeltaThreshold, Clean,Remove_temps)
        #res[scene['name']] = dfclone['any'].any()
        overall_df.loc[fall_file,scene['name']] = dfclone['any'].any() * 1
    #non_fall_results.append(res)

In [8]:
overall_df['event'] = (overall_df['event'] == 'fall') * 1
for scene in scenes:
    overall_df[scene['name']] = overall_df[scene['name']].astype('int')

## View File Tag and Event

In [9]:
@interact(is_file = IntSlider(min = 0 , max = filecount - 1, description='FILE'))
def view_row(is_file):
    return pd.DataFrame(overall_df.iloc[is_file])

interactive(children=(IntSlider(value=0, description='FILE', max=483), Output()), _dom_classes=('widget-intera…

In [10]:
# https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_display_object_visualization.html#sphx-glr-auto-examples-miscellaneous-plot-display-object-visualization-py

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

clf = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))


In [16]:
@interact( 
    cb_nfs = Checkbox(description='normal fall scene'),
    cb_ffs = Checkbox(description='frontal fall scene'),
    cb_cfs1 = Checkbox(description='chair fall scene 1'),
    cb_cfs2 = Checkbox(description='chair fall scene 2'),
    cb_cfs3 = Checkbox(description='chair fall scene 3'),
    cb_nefs = Checkbox(description='near fall scene'),
    cb_dfs1 = Checkbox(description='diagonal fall scene 1'),
    cb_dfs2 = Checkbox(description='diagonal fall scene 2'),
    cb_ws = Checkbox(description='walking scene'),
    cb_wps = Checkbox(description='walking picking scene'),
    cb_wss = Checkbox(description='walking sitting scene'),
    cb_fws = Checkbox(description='frontal walking scene'),
    cb_fwps = Checkbox(description='frontal walking picking scene'),
    cb_bs = Checkbox(description='bed scene'),
    dd_scene = Dropdown(description= 'target scene',options = [sc['name'] for sc in scenes])
    
)
def check(
    cb_nfs,
    cb_ffs,
    cb_cfs1,
    cb_cfs2,
    cb_cfs3,
    cb_nefs,
    cb_dfs1,
    cb_dfs2,
    cb_ws,
    cb_wps,
    cb_wss,
    cb_fws,
    cb_fwps,
    cb_bs,
    dd_scene
):
    sel_scenes = []
    if cb_nfs : sel_scenes.append('normal fall scene')
    if cb_ffs : sel_scenes.append('frontal fall scene')
    if cb_cfs1 : sel_scenes.append('chair fall scene 1')
    if cb_cfs2 : sel_scenes.append('chair fall scene 2')
    if cb_cfs3 : sel_scenes.append('chair fall scene 3')
    if cb_nefs : sel_scenes.append('near fall scene')
    if cb_dfs1 : sel_scenes.append('diagonal fall scene 1')
    if cb_dfs2 : sel_scenes.append('diagonal fall scene 2')
    if cb_ws : sel_scenes.append('walking scene')
    if cb_wps : sel_scenes.append('walking picking scene')
    if cb_wss : sel_scenes.append('walking sitting scene')
    if cb_fws : sel_scenes.append('frontal walking scene')
    if cb_fwps : sel_scenes.append('frontal walking picking scene')
    if cb_bs : sel_scenes.append('bed scene')
        
    if len(sel_scenes) == 0 : return 'Select at least 1 scene'
    
    scene_filter = overall_df['scene'] == 0
    
    for ss in sel_scenes:
        scene_filter |= overall_df['scene'] == ss
    
    target_rows = overall_df[scene_filter]
    
    
    y_true = target_rows['event']
    
    # print(y_true)
    
    y_pred = target_rows[dd_scene]
    
    cm = confusion_matrix(y_true, y_pred)

    cm_display = ConfusionMatrixDisplay(cm).plot()
    
    
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
    
    prec, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

interactive(children=(Checkbox(value=False, description='normal fall scene'), Checkbox(value=False, descriptio…